# Naive Strategy - 6 Class Grouped

In [1]:
#basic imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler as MMS
#torch
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn import MSELoss
from torch.nn import BCEWithLogitsLoss,BCELoss

#avalanche
from avalanche.models import SimpleMLP,MlpVAE,SlimResNet18,IcarlNet #model
from avalanche.training.supervised import Naive,Cumulative,Replay,LwF,ICaRL #cl strat
from avalanche.benchmarks.generators import nc_benchmark,ni_benchmark #custom benchmark
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics #metrics
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, CSVLogger #loggers
from avalanche.training.plugins import EvaluationPlugin


d:\anaconda\envs\CLFL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_train=pd.read_csv('../data/balanced_dataset.csv')

In [3]:
df_train.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,fiveLabels
0,53,30931.0,2,2,58,122,29,29,29.000000,0.000000,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
1,53,31324.0,2,2,84,228,42,42,42.000000,0.000000,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
2,53,178.0,2,2,64,96,32,32,32.000000,0.000000,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
3,443,146835.0,9,10,520,4580,194,0,57.777778,75.057941,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
4,36030,3.0,2,0,0,0,0,0,0.000000,0.000000,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0


# Load Data

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
# Splitting the DataFrame into train and test sets
train_df, test_df = train_test_split(df_train, test_size=0.2, random_state=42)



In [6]:
x_train=train_df.drop(['Label','fiveLabels'],axis=1)
y_train=train_df.fiveLabels
x_test=test_df.drop(['Label','fiveLabels'],axis=1)
y_test=test_df.fiveLabels

print(x_train.shape,x_test.shape,y_train.shape)
x_train=x_train.values
x_test=x_test.values
y_train=y_train.values
y_test=y_test.values

(25152, 70) (6288, 70) (25152,)


In [7]:

mms=MMS(feature_range=(0,1))
x_train=mms.fit_transform(x_train)
x_test=mms.fit_transform(x_test)
x_train=torch.tensor(x_train,dtype=torch.float32)
y_train=torch.tensor(y_train)
x_test=torch.tensor(x_test,dtype=torch.float32)
y_test=torch.tensor(y_test)

In [8]:
type(x_train)

torch.Tensor

In [9]:
train_dataset=TensorDataset(x_train,y_train)
test_dataset=TensorDataset(x_test,y_test)
NC_benchmark = nc_benchmark(train_dataset=train_dataset,test_dataset = test_dataset,n_experiences=6,task_labels=False,shuffle=False)
#NI_benchmark=ni_benchmark(train_dataset=train_dataset,test_dataset = test_dataset,n_experiences=1,task_labels=False,seed=1234)

In [10]:
x_train[0].dim(),y_train.dim()

(1, 1)

In [11]:
model_2=SimpleMLP(num_classes=6,input_size=70)

In [12]:
text_logger = TextLogger(open('../logs/naive5.txt', 'a'))
interactive_logger = InteractiveLogger()
csv_logger=CSVLogger("../logs/naive5.csv")



# # Strategy
# cl_strategy = Naive(
#     model=model_2, optimizer=Adam(model_2.parameters(), lr=0.001),
#     criterion=CrossEntropyLoss(), train_mb_size=256, train_epochs=1, eval_mb_size=300,
#     evaluator=eval_plugin)

In [13]:
y_train[0].size()

torch.Size([])

In [27]:
import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from avalanche.training import (
    Cumulative, JointTraining, Naive, AR1, StreamingLDA, ICaRL, PNNStrategy,
    CWRStar, Replay, GSS_greedy, GDumb, LwF, AGEM, GEM, EWC, SynapticIntelligence,
    CoPE, LFL, GenerativeReplay, MAS, BiC, MIR, MER, LearningToPrompt, SCR,
    FromScratchTraining, ExpertGateStrategy, DER
)
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics
from avalanche.logging import TextLogger

# Assuming model_2 is already defined
optimizer = Adam(model_2.parameters(), lr=0.001)
optimizer2 = Adam(model_2.parameters())
criterion = CrossEntropyLoss()
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    #loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    #timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    #confusion_matrix_metrics(num_classes=NC_benchmark.n_classes, save_image=False, stream=True),
    loggers=[interactive_logger, text_logger,csv_logger]
)

# Common parameters
train_mb_size = 256
train_epochs = 1
eval_mb_size = 300

# Additional parameters for specific strategies
mem_size = 1000  # Memory size for strategies that need it
ewc_lambda = 0.4  # Regularization term for EWC
alpha = 0.5  # Used in strategies like LwF

# Define strategies
strategies = {
    "Cumulative": Cumulative(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "JointTraining": JointTraining(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "Naive": Naive(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "Replay": Replay(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "GDumb": GDumb(model_2, optimizer, criterion, mem_size, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "LwF": LwF(model_2, optimizer, criterion, alpha, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "GEM": GEM(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "SynapticIntelligence": SynapticIntelligence(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "MIR": MIR(model_2, optimizer, criterion, mem_size, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "MER": MER(model_2, optimizer, criterion, mem_size, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "FromScratchTraining": FromScratchTraining(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    "DER": DER(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),

    #NOT USING:
    #"CWRStar": CWRStar(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"AR1": AR1(model_2, optimizer2, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"StreamingLDA": StreamingLDA(slda_model, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"ICaRL": ICaRL(feature_extractor, classifier, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin, memory_size=mem_size),
    #"PNNStrategy": PNNStrategy(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"GSS_greedy": GSS_greedy(model_2, optimizer, criterion, mem_size, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"EWC": EWC(model_2, optimizer, criterion, ewc_lambda, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"LFL": LFL(model_2, optimizer, criterion, lambda_e, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"MAS": MAS(model_2, optimizer, criterion, lambda_reg, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"LearningToPrompt": LearningToPrompt(model_name, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),  # 'model_name' needs to be defined
    #"SCR": SCR(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"ExpertGateStrategy": ExpertGateStrategy(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"AGEM": AGEM(model_2, optimizer, criterion, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"CoPE": CoPE(model_2, optimizer, criterion, mem_size, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"GenerativeReplay": GenerativeReplay(model_2, optimizer, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),
    #"BiC": BiC(model_2, optimizer, criterion, mem_size, train_mb_size, train_epochs, eval_mb_size, evaluator=eval_plugin),

}

In [28]:
def train_and_evaluate(strategy, benchmark, strategy_name):
    results = []
    if strategy_name == "JointTraining":
        # Train on the entire stream for JointTraining
        strategy.train(benchmark.train_stream)
        res = strategy.eval(benchmark.test_stream)
        results.append(res)
    else:
        # For other strategies, train on each experience
        for experience in benchmark.train_stream:
            strategy.train(experience)
            res = strategy.eval(benchmark.test_stream)
            results.append(res)
    return results

# Dictionary to store results
strategy_results = {}

# Train and evaluate each strategy
for strategy_name, strategy in strategies.items():
    print("\n\n\n-----------------------------------------")
    print(f"Training with {strategy_name}")
    print("-----------------------------------------\n\n\n")
    results = train_and_evaluate(strategy, NC_benchmark, strategy_name)
    strategy_results[strategy_name] = results
    print(f"Completed training with {strategy_name}")

# Write results to a file
with open('strategy_results.txt', 'w') as file:
    for strategy_name, results in strategy_results.items():
        file.write(f"{strategy_name}: {results}\n")

print("All strategies have been evaluated. Results are saved in 'strategy_results.txt'")





-----------------------------------------
Training with Cumulative
-----------------------------------------



-- >> Start of training phase << --
943it [02:55,  3.08s/it]                           

958it [02:55,  5.45it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 1.0000
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 4/4 [00:00<00:00, 66.67it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 1.0000
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 4/4 [00:00<00:00, 67.80it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 4/4 [00:00<00:00, 66.67it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.0000
-- Starting eval on experience 3 (Task 0) from test stream --
100%|██████████| 4/4 [00:00<00:00, 67.80it/s

In [ ]:

print('Starting experiment...')
results = []
for experience in NC_benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(NC_benchmark.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0]
-- >> Start of training phase << --
100%|██████████| 17/17 [00:00<00:00, 47.89it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.3910
	Loss_MB/train_phase/train_stream/Task000 = 0.9288
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0017
	Time_Epoch/train_phase/train_stream/Task000 = 0.3570
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8400
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 4/4 [00:00<00:00, 48.19it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.8716
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 1.0000
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 4/4 [00:00<0

Training with Cumulative
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


RuntimeError: shape '[256, 70]' is invalid for input of size 200704